In [41]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import os
from glob import glob
import time
import zipfile
import plotly.graph_objects as go
import plotly.express as px

In [42]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [43]:
anio=str(input('desde que anio: '))

In [44]:
carpeta_descarga=os.getcwd().replace('src','downloads')
#Con getcwd() se encuentra el path absoluto

def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
chrome_options.add_experimental_option('prefs', prefs)

service = Service(ChromeDriverManager().install())

# Crear el objeto WebDriver con el objeto Service
driver = webdriver.Chrome(service=service, options=chrome_options)

link_consulta='https://comex.indec.gob.ar/?_ga=2.1947448.887743671.1661390997-1687177890.1630124319#/database'
driver.get(link_consulta)

lista_botones=driver.find_elements(By.CLASS_NAME, "form-control")

#A partir de acá, quiero  cambiar el anio hacia delante

select_comercio=Select(lista_botones[0])
select_comercio.select_by_value('imports')

select_frecuencia=Select(lista_botones[2])
select_frecuencia.select_by_visible_text('Mensual')

#Lo pongo despues para hacer el loop

select_anio=Select(lista_botones[1])


opciones=[]
for option in select_anio.options: 
    opciones.append(option.get_attribute('value'))
opciones.pop(0)
# opciones=filter(lambda x: int(x)>=int(anio),opciones)
opciones=list(filter(lambda x: x>=anio,opciones))

for opcion in opciones:
    select_anio.select_by_visible_text(opcion)
    descarga=driver.find_element(By.CLASS_NAME, "btn-outline-primary")
    descarga.click()
    time.sleep(3)

# concatenacion_dfs()
paths = WebDriverWait(driver, 300, 1).until(every_downloads_chrome)

# for i in glob("../downloads/*", recursive = True): os.remove(i)
driver.quit()

In [45]:
columnas=['Año', 'Mes', 'NCM', 'Porg', 'Pnet(kg)', 'FOB(u$s)', 'Flete(u$s)',
       'Seguro(u$s)', 'CIF(u$s)']
def concatenacion_dfs(opciones=opciones):
    '''Extrae y concatena las dfs'''
    dfs={}
    for idx, opcion in enumerate(opciones):
        with zipfile.ZipFile(f'../downloads/imports_{opcion}_M.zip', 'r') as zip_ref:
            zip_ref.extractall('../downloads')
        df=pd.read_csv(f'../downloads/impom{opcion[2:]}.csv', sep=';', encoding='latin-1',decimal=',', dtype={'NCM':str, 'Porg':str})
        df.rename(columns={col: col.lower() for col in columnas}, inplace=True)
        # df['Año'] = df.Año.astype(str)
        # df['mes'] = df.Mes.astype(str)
        df.ncm=df.ncm.apply(lambda x: x.strip())
        dfs[str(2022-idx)] = df
    return pd.concat([dfs[key] for key in dfs.keys()])

df_completa=concatenacion_dfs()
soja_desagregado=pd.read_excel('../downloads/impo_soja_y_sub_productos_mens_2002_2022.xlsx',sheet_name=1)

In [46]:
#lista de paises
paises=pd.read_csv('../data/CE_PAIS.csv', sep=';', dtype={'CCOD_ASOC':str})
paises['cod_final']=paises.CCOD_ASOC.fillna(paises.CCOD_PAIS)
dic_paises=dict(zip(paises.CCOD_PAIS,paises.CDESCRI))
paises['descri_final'] = paises.cod_final.apply(lambda x: dic_paises[x])
dic_cod_paises=dict(zip(paises.CCOD_PAIS,paises.cod_final))
asociados=paises[~paises.CCOD_ASOC.isna()] #para ver cod con asociados
dic_paises_asociados=dict(zip(paises.CCOD_PAIS,paises.descri_final))
paises=paises[['cod_final','descri_final']]
#Paises
df_completa.insert(4,'pais_descri',df_completa.porg.apply(lambda x: dic_paises[x]))
df_completa.insert(5,'cod_final',df_completa.porg.apply(lambda x: dic_cod_paises[x]))
df_completa.insert(6,'descri_final',df_completa.porg.apply(lambda x: dic_paises_asociados[x]))
df_completa=df_completa.fillna(0)
df_completa = df_completa.drop('porg',axis=1)
df_completa = df_completa.drop('pais_descri', axis= 1)
df_completa.rename({'cod_final':'cod_origen','descri_final':'pais_descri'},axis = 1,inplace = True)

In [47]:
#Para ver el ultimo mes disponible
dic_meses={
    'Enero':1,
    'Febrero':2,
    'Marzo':3,
    'Abril':4,
    'Mayo':5,
    'Junio':6,
    'Julio':7,
    'Agosto':8,
    'Septiembre':9,
    'Octubre':10,
    'Noviembre':11,
    'Diciembre':12
}
dic_meses={v:k for k,v in dic_meses.items()}
ultimo_mes_disponible = dic_meses[int(df_completa.sort_values(['año','mes'],ascending=False).reset_index(drop=True)
         .mes.iloc[0])]
ultimo_anio_disponible = int(df_completa.sort_values(['año','mes'],ascending=False).reset_index(drop=True)
         .año.iloc[0])

ultimo_mes_disponible

'Febrero'

In [48]:
descripciones_df = pd.read_csv('../data/CE_ENMIENDA.csv',sep=';',dtype={'ncm':str,'cnro_enmienda':int})
def put_descripciones_ncm(df):
    return df.merge(descripciones_df, on = 'ncm',how = 'left')

df_completa = put_descripciones_ncm(df_completa)

In [49]:
df_anual = df_completa.groupby(['año','ncm','ncm_descri','cod_origen','pais_descri','cnro_enmienda'],as_index=False).sum()

In [50]:
df_completa.to_csv("../output/impo_desagregado_ncm_pais.csv")
df_completa.groupby(['año','mes'], as_index=False).sum().to_csv('../output/impo_anual.csv')

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\1791722285.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [51]:
dic_meses={
    'Enero':1,
    'Febrero':2,
    'Marzo':3,
    'Abril':4,
    'Mayo':5,
    'Junio':6,
    'Julio':7,
    'Agosto':8,
    'Septiembre':9,
    'Octubre':10,
    'Noviembre':11,
    'Diciembre':12
}
dic_meses={v:k for k,v in dic_meses.items()}

def soja_plot(df=soja_desagregado, variable = "cif"):
    if variable == "cif":
        variable = "CIF(dol)"
        titulo = "millones de USD"
    elif variable == "pnet":
        variable = "pnet(kg)"
        titulo = "toneladas"
        
    año_final=df.año.iloc[-1]
    mes_final=df.mes.iloc[-1]
    try:
        top_5=(df.groupby('porg',as_index=False).sum()
        .sort_values('CIF(dol)',ascending=False)).porg[:5].to_list()
    except: 
        df.rename({"pporg":"porg"},axis=1,inplace=True)
        top_5=(df.groupby('porg',as_index=False).sum()
        .sort_values('CIF(dol)',ascending=False)).porg[:5].to_list()

    df=df[df.porg.isin(top_5)].reset_index(drop=True)
    df=df.groupby(['año','porg'],as_index=False).sum()
    df.rename({'porg':'País de origen'},axis=1,inplace=True)
    df["CIF(dol)"] = df["CIF(dol)"]/1000000
    df["pnet(kg)"] = df["pnet(kg)"]/1000
             
    ncm_cif=px.bar(df, x=df['año'].astype(str),y=df[variable],
    color="País de origen", title=f"Importaciones de poroto de soja<br><sup>En {titulo}</sup>")

    # ncm_cif.update_xaxes(showgrid=True, title_text="Mes", title_standoff=7)
    ncm_cif.update_yaxes(tickformat=',', title_text = "")
    ncm_cif.update_xaxes(title_text="")
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    xaxis = dict(tickmode = "linear"),
    template = 'none',
    # title_text=f"CIF importado de :<br>\"Poroto de soja\"<br><sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1.07, xanchor="left", x=0.1, orientation="h"))
    note = f'Fuente: MartinBasualdo0 en base a INDEC. Top 5 países, datos del {año_final} hasta {dic_meses[mes_final]}'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.57, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_cif
soja_plot(variable="pnet")

# soja_desagregado

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\3171114220.py:32: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\3171114220.py:36: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [52]:
def titles_nonan(title):
    for x in title:
        if type(x)!=float: return x
        
def get_lista_paises(dfs: list):
    titles= [dfs[i]["pais_descri"].unique() for i in range(len(dfs)) ]
    return [ titles_nonan(title) for title in titles]

def ncm_descri_nonan(title):
    for x in title:
        if type(x)!=float: return x
 
def acorta_descri(descri, largo = 60):
    if len(descri)>=largo:
        return descri[:largo]+'...'
    else:
        return descri
            
def crea_tablas_ncm_mensual(df,ncm, desde_anio):
    ncm = str(ncm)

    df=df.sort_values(['año','mes'],ascending=True).reset_index(drop=True)
    df = df[df.año >= desde_anio]
    año_final=df.año.iloc[-1]
    mes_final=df.mes.iloc[-1]
    hasta=str(mes_final+1)+'/'+'1'+'/'+str(año_final)
    desde=str(df.mes.iloc[0])+'/'+'1'+'/'+str(df.año.iloc[0])
    
    df = df[df.ncm == ncm]
    df['fecha'] = pd.to_datetime(df['mes'].astype(str)+'/'+df['año'].astype(str),format = "%m/%Y")
    df.fecha = df.fecha.dt.strftime('%m/%Y')
    
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='M').strftime('%m/%Y')
    })
        
    dfs=[pd.merge(left=monthDates,right=i, on='fecha',how='outer') for x, i in df.groupby(['pais_descri'], as_index=False)]
        
    return dfs

def crea_tablas_ncm_anual(df,ncm, desde_anio):
    ncm = str(ncm)
    df=df.sort_values(['año'],ascending=True).reset_index(drop=True)
    df = df[df.año >= desde_anio]
    año_final=df.año.iloc[-1]
    hasta='1'+'/'+'1'+'/'+str(año_final)
    desde="1"+'/'+'1'+'/'+str(df.año.iloc[0])
    
    df = df[df.ncm == ncm]
    df['fecha'] = pd.to_datetime(df['año'].astype(str),format = "%Y")
    df.fecha = df.fecha.dt.strftime('%Y')
    
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='Y').strftime('%Y')
    })
        
    dfs=[pd.merge(left=monthDates,right=i, on='fecha',how='outer') for x, i in df.groupby(['pais_descri'], as_index=False)]
        
    return dfs
crea_tablas_ncm_anual(df_completa,'12019000',2008)[0].columns

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\1709330327.py:57: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



Index(['fecha', 'año', 'mes', 'ncm', 'cod_origen', 'pais_descri', 'pnet(kg)',
       'fob(u$s)', 'flete(u$s)', 'seguro(u$s)', 'cif(u$s)', 'cnro_enmienda',
       'ncm_descri'],
      dtype='object')

In [53]:
def ncm_cif(dfs):

    titles = get_lista_paises(dfs)
             
    ncm_cif=go.Figure()
    for i in range(len(dfs)):
        x=dfs[i]['fecha']
        ncm_cif.add_trace(go.Bar(x=x, y=dfs[i]["cif(u$s)"]/1000000, name=titles[i], textposition='inside',
        # text=dfs[i]["cif_licuado"]/1000, texttemplate = '%{text:.2f}'
        ))

    # ncm_cif.update_xaxes(showgrid=True, title_text="Mes", title_standoff=7)
    ncm_cif.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    # xaxis = dict(tickmode = "linear"),
    template = 'none',
    title_text=f"CIF importado de :<br>\"{acorta_descri(ncm_descri_nonan(dfs[0].ncm_descri))}\"<br>NCM: {ncm_descri_nonan(dfs[0].ncm)}<br> <sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1, xanchor="left", x=1, orientation="v"))
    note = f'Fuente: MartinBasualdo0 en base a INDEC. Top 5 países, datos del {ultimo_anio_disponible} hasta {ultimo_mes_disponible}'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.57, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_cif

ncm_cif(crea_tablas_ncm_mensual(df_completa,'64029990',2016))

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\1709330327.py:37: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



In [54]:
def cif_ncm_mensual_pais(df=df_completa, ncm = '27111100', anio_desde = 2019):
    ncm=str(ncm)
    df.rename({'pais_descri':'País de origen'},axis=1,inplace=True)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)
    
    df=df.sort_values(['año','mes'],ascending=True).reset_index(drop=True)
    año_final=df.año.iloc[-1]
    mes_final=df.mes.iloc[-1]
    hasta=str(mes_final+1)+'/'+'01'+'/'+str(año_final)
    desde=str(df.mes.iloc[0])+'/'+'01'+'/'+str(df.año.iloc[0])

    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0])
    df.mes=df.mes.astype(int)
    df = df.sort_values(['año','mes'],ascending=True)
    df.mes=df.mes.astype(str)
    df.año = df.año.astype(str)
    df['fecha']=pd.to_datetime(df['mes']+'/'+df['año'],format = "%m/%Y")
        
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='M').strftime('%m-%Y')
    })
    
             
    ncm_cif=px.bar(df, x=df["fecha"],y=df["cif(u$s)"]/1000000,
    color="País de origen", title=f"CIF importado de <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    category_orders={'fecha':monthDates.fecha},)
    
    ncm_cif.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_cif.update_xaxes(title_text="",dtick="M1",tickformat="%m/%Y")
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    # xaxis = dict(tickmode = "linear"),
    template = 'none',
    # title_text=f"CIF importado de :<br>\"Poroto de soja\"<br><sup>En millones de dólares</sup>",
    # legend = dict(yanchor="top",y=1.07, xanchor="left", x=0.1, orientation="h")
    )
    note = f'Fuente: MartinBasualdo0 en base a INDEC'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.3, yref='paper', y=-0.18,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_cif,df

plot,df=cif_ncm_mensual_pais()
plot

In [55]:
def cif_ncm_anual_pais(anio_desde,df=df_anual, ncm = '12019000'):
    ncm=str(ncm)
    df.rename({'pais_descri':'País de origen'},axis=1,inplace=True)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)
    
    df=df.sort_values(['año'],ascending=True).reset_index(drop=True)
    año_final=df.año.iloc[-1]
    hasta=str(1)+'/'+'01'+'/'+str(año_final)
    desde=str(1)+'/'+'01'+'/'+str(df.año.iloc[0])

    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0])
    df = df.sort_values(['año'],ascending=True)
    df.año = df.año.astype(str)
    df['fecha']=pd.to_datetime(df['año'],format = "%Y")
        
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='Y').strftime('%Y')
    })
             
    ncm_cif=px.bar(df, x=df["año"],y=df["cif(u$s)"]/1000000,
    color="País de origen", title=f"CIF importado de <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    category_orders={'año':df.año},
    )
    
    ncm_cif.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_cif.update_xaxes(title_text="", type='category')
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    # xaxis = dict(tickmode = "linear"),
    template = 'none',
    # title_text=f"CIF importado de :<br>\"Poroto de soja\"<br><sup>En millones de dólares</sup>",
    # legend = dict(yanchor="top",y=1.07, xanchor="left", x=0.1, orientation="h")
    )
    note = f'Fuente: MartinBasualdo0 en base a INDEC. Top 5 países, datos del {ultimo_anio_disponible} hasta {ultimo_mes_disponible}'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.6, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_cif

cif_ncm_anual_pais(anio_desde=2008,ncm = '12019000')

In [65]:
def ncm_fob_anual(anio_desde,df=df_anual, ncm='27111100'):
    ncm=str(ncm)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)
    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0], largo = 60)
    df=df.groupby('año',as_index=False).sum()
    # titles = get_lista_paises(dfs)
             
    ncm_fob=go.Figure()
    # for i in range(len(dfs)):
    x=df['año']
    ncm_fob.add_trace(go.Bar(x=x, y=df["cif(u$s)"]/1000000, name="FOB Aceite de soja", textposition='inside',
    # text=dfs[i]["fob_licuado"]/1000, texttemplate = '%{text:.2f}'
    ))

    # ncm_fob.update_xaxes(showgrid=True, title_text="Mes", title_standoff=7)
    ncm_fob.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_fob.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_fob.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    xaxis = dict(tickmode = "linear"),
    template = 'none',
    title_text=f"CIF importado de: <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1, xanchor="left", x=1, orientation="v"))
    note = f'Fuente: @MartinBasualdo0 en base a INDEC. Datos del {ultimo_anio_disponible} hasta {ultimo_mes_disponible}'
    ncm_fob.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.5, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_fob

ncm_fob_anual(2008, ncm="64041900")


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\810875667.py:6: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [57]:
def ncm_cif_mensual(df=df_completa, ncm = '27111100',anio_desde = 2019):
    ncm=str(ncm)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)

    df=df.sort_values(['año','mes'],ascending=True).reset_index(drop=True)
    año_final=df.año.iloc[-1]
    mes_final=df.mes.iloc[-1]
    hasta=str(int(mes_final)+1)+'/'+'01'+'/'+str(año_final)
    desde=str(df.mes.iloc[0])+'/'+'01'+'/'+str(df.año.iloc[0])
    

    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0], largo = 60)
    df=df.groupby(['año','mes'],as_index=False).sum()
    df.mes=df.mes.astype(int)
    df=df.sort_values(['año','mes'],ascending=True)
    df.mes=df.mes.astype(str)
    df.año=df.año.astype(str)
    df['fecha']=pd.to_datetime(df['mes']+'/'+df['año'],format = "%m/%Y")
        
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='M').strftime('%m-%Y')
    })
    
    ncm_cif=px.bar(df, x=df.fecha,y=df["cif(u$s)"]/1000000,
    title=f"CIF importado de <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    category_orders={'fecha':monthDates.fecha}, )

    ncm_cif.update_xaxes(title_text="",dtick="M1",tickformat="%m/%Y")
    ncm_cif.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", font_family="Georgia", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    # xaxis = dict(tickmode = "linear"),
    template = 'none',
    # title_text=f"CIF importado de :<br>\"Poroto de soja\"<br><sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1.07, xanchor="left", x=0.0, orientation="h"))
    note = f'Fuente: MartinBasualdo0 en base a INDEC'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.26, yref='paper', y=-0.18,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    
    
    return ncm_cif

ncm_cif_mensual()

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_19220\566533480.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [58]:
for i in glob("../downloads/*", recursive = True): os.remove(i)